## How to use the BoTorch (quasi-) Monte-Carlo based acquisition functions in NEMO

The (quasi-) Monte-Carlo based acquisition functions in the BoTorch library has demonstrated excellent Bayesian optimisation performance with short wall-times for suggesting new candidates. However, they are not used by default for NEMO's expected improvement based methods because NEMO uses various types of regression models and calculable objectives that do not all have a `posterior` method and do not allow back-propagating gradients from targets back to the inputs. This precludes the use of BoTorch's methods for all of NEMO's native machine learning models except for Gaussian processes (GPs).

As the GPs used in NEMO are derived from the BoTorch library, a NEMO optimisation problem that exclusively uses GP models can be exploited to use the `qNoisyExpectedImprovement` and `qExpectedHypervolumeImprovement` acquisition functions from the BoTorch library.

This tutorial will demonstrate how to set up an optimisation that can utilise these.

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

In [ ]:
# Create the variable objects
var1 = ContinuousVariable(name="variable1", lower_bound=0.0, upper_bound=100.0)
var2 = ContinuousVariable(name="variable2", lower_bound=0.0, upper_bound=100.0)
var_list = VariablesList([var1, var2])

### Specifying the machine learning model types for objectives to be GPs

By specifying the objectives to be GPs, this guarantees that the models will be compatible with the BoTorch methods. 

In [ ]:
# Create the objective objects
obj1 = RegressionObjective(
    name="objective1",
    obj_max_bool=True,
    lower_bound=0.0,
    upper_bound=100.0,
    predictor_type="gp",
)
obj2 = RegressionObjective(
    name="objective2",
    obj_max_bool=False,
    lower_bound=0.0,
    upper_bound=100.0,
    predictor_type="gp",
)
obj_list = ObjectivesList([obj1, obj2])

In [ ]:
# Instantiate the sampler
sampler = LatinHyperCubeSampling()

The `force_botorch_ei_methods` keyword argument defines whether to force the expected improvement type methods to use the BoTorch-based ones, as long as the following conditions are met: 
    1. All objectives are modelled with GPs
    2. All constraints, if used, are LinearConstraint objects
    3. The sampler chosen is not a PoolBased sampler

Please note that above we forced the objectives to be modelled using GPs. Conversely, if we allowed the objectives to take on different machine learning models with `force_botorch_ei_methods` still set to `True`, the BoTorch methods can still be used without forcing for GP models as long as GPs are found to be the best models. If during an optimisation run, the best models varies and is not a GP, then the NEMO-based expected improvement methods are used automatically without raising an Exception.

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4, force_botorch_ei_methods=True)

In [ ]:
# Set up the optimisation instance
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler)

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
# X and Y arrays represent a hypothetical initial dataset
optimisation_data = optimisation.run(X, Y, number_of_iterations=50)